In [1]:
import numpy as np
from numpy import random as rd
from pykrx import stock

#아래 티커는 각각 kodex레버리지 / KODEX 인버스 / kosef미국달러선물 / KODEX 모멘텀PLUS
ETF_ticker = ["122630", "114800", "138230", "244620"]
N = len(ETF_ticker) #ETF 갯수만큼 자산 총 갯수는 N개로 제한된다.
stocks_data = []
srt = "20140101"
end = "20191231"
for i in range(0, N):
    stocks_data.append(stock.get_market_ohlcv_by_date(srt, end, ETF_ticker[i])["종가"][-300:])#룩백윈도우는 300일


In [6]:
pd.DataFrame(stocks_data)

날짜,2018-10-15,2018-10-16,2018-10-17,2018-10-18,2018-10-19,2018-10-22,2018-10-23,2018-10-24,2018-10-25,2018-10-26,...,2019-12-16,2019-12-17,2019-12-18,2019-12-19,2019-12-20,2019-12-23,2019-12-24,2019-12-26,2019-12-27,2019-12-30
종가,12605,12555,12870,12630,12730,12795,12185,12070,11700,11320,...,13690,14065,14050,14080,14145,14125,13930,14045,14485,14425
종가,6930,6950,6850,6915,6890,6875,7075,7100,7205,7325,...,6510,6425,6435,6420,6410,6410,6460,6435,6330,6350
종가,11720,11655,11630,11715,11690,11665,11745,11700,11760,11800,...,12395,12305,12335,12315,12270,12305,12300,12280,12275,12230
종가,9850,9969,10098,9949,10028,10093,9720,9467,9264,9011,...,8780,8885,8900,8920,8960,8940,8915,9070,9255,9315


In [2]:
rf = 0.015 #벤치마크(무위험)수익률은 1.5%로 고정되어있다고 가정.
w = [1/N for i in range(0,N)]

In [5]:
import pandas as pd

ret = pd.DataFrame(np.nan,index=np.arange(len(stocks_data[0])), columns=ETF_ticker)
for stock, idx in zip(ETF_ticker, range(0,N)):
    for i in range(1, len(stocks_data[0])):
        ret[stock].iloc[i] =  stocks_data[idx][i]/stocks_data[idx][i-1] 
        
retArr = []
for stock in ETF_ticker:
    retArr.append( ret[stock].iloc[1:] )# 0일 수익률은 nan 값이라서 날짜 체크할때 하루 밀리는거 조심할것.

In [4]:
for i in range(4):
    print(np.std(retArr[i]))

0.017742209433987587
0.0090300418897901
0.003957478529488991
0.010646568306214856


In [5]:
stocks_data[2]

날짜
2018-10-15    11720
2018-10-16    11655
2018-10-17    11630
2018-10-18    11715
2018-10-19    11690
              ...  
2019-12-23    12305
2019-12-24    12300
2019-12-26    12280
2019-12-27    12275
2019-12-30    12230
Name: 종가, Length: 300, dtype: int32

In [6]:
retArr[0]

1      0.996033
2      1.025090
3      0.981352
4      1.007918
5      1.005106
         ...   
295    0.998586
296    0.986195
297    1.008256
298    1.031328
299    0.995858
Name: 122630, Length: 299, dtype: float64

In [7]:
np.cumprod(retArr[0])

1      0.996033
2      1.021023
3      1.001983
4      1.009917
5      1.015073
         ...   
295    1.120587
296    1.105117
297    1.114240
298    1.149147
299    1.144387
Name: 122630, Length: 299, dtype: float64

In [8]:
np.cumprod([1, 1.1 , 1.3, 0.9, 0.9])

array([1.    , 1.1   , 1.43  , 1.287 , 1.1583])

In [9]:
np.cov(retArr[0], retArr[1])

array([[ 3.15842324e-04, -1.60133892e-04],
       [-1.60133892e-04,  8.18152863e-05]])

In [29]:
retArr[0]

1      0.996033
2      1.025090
3      0.981352
4      1.007918
5      1.005106
         ...   
295    0.998586
296    0.986195
297    1.008256
298    1.031328
299    0.995858
Name: 122630, Length: 299, dtype: float64

In [30]:
len(retArr)

4

In [10]:
#var - cov matrix 생성
def getCov(a, b):
    return np.cov(a,b)

var_cov_mat = np.identity(N) # N * N 단위행렬
for i in range(0, N):
    for j in range(0, N):
        var_cov_mat[i][j] = getCov(retArr[i], retArr[j])[0,1]

In [32]:
#RETURN VAR - COV MATRIX
def getCovMat(returns):
    N = len(returns)
    var_cov_mat = np.identity(N)
    for i in range(0,N):
        for j in range(0,N):
            var_cov_mat[i][j] = getCov(returns[i],returns[j])[0,1]
    return var_cov_mat

In [11]:
var_cov_mat

array([[ 3.15842324e-04, -1.60133892e-04, -4.35816222e-05,
         1.22531038e-04],
       [-1.60133892e-04,  8.18152863e-05,  2.20628287e-05,
        -6.27481997e-05],
       [-4.35816222e-05,  2.20628287e-05,  1.57141921e-05,
        -1.95184371e-05],
       [ 1.22531038e-04, -6.27481997e-05, -1.95184371e-05,
         1.13729784e-04]])

In [35]:
import scipy as sp
from scipy import optimize

def eq_constraint(w): # w값들의 합이 1이 되어야 한다.
    summation = 0
    for i in w:
        summation += i
    return summation -1

def flarray(w): #샤프지수가 최대가 되게하는 목적함수
    ret_exd = []
    for i in range(N):
        ret_exd.append(float(np.cumprod(retArr[i])[-1:] - rf))
#    print(ret_exd, '초과수익률')
#    print(np.dot(np.transpose(ret_exd), w), '분자')
#    print(np.dot(np.dot(np.transpose(w), var_cov_mat), w)**(1/2), '분모')
    return -(np.dot(np.transpose(ret_exd), w) / (np.dot(np.dot(np.transpose(w), var_cov_mat), w))**(1/2))


In [36]:
def get_W_Max_Sharp():
    return sp.optimize.fmin_slsqp(flarray, w_init, bounds=[(0,1) for i in range(N)], eqcons=[eq_constraint])

In [13]:
float(np.cumprod(retArr[0])[-1:] - rf)

1.1293871479571596

In [78]:
np.cumprod(retArr[0])

1      0.996033
2      1.021023
3      1.001983
4      1.009917
5      1.015073
         ...   
295    1.120587
296    1.105117
297    1.114240
298    1.149147
299    1.144387
Name: 122630, Length: 299, dtype: float64

In [72]:
retArr[0]

1      0.996033
2      1.025090
3      0.981352
4      1.007918
5      1.005106
         ...   
295    0.998586
296    0.986195
297    1.008256
298    1.031328
299    0.995858
Name: 122630, Length: 299, dtype: float64

In [16]:
w_init = [1/N for i in range(0,N)] # 초기값은 동일비중[1/4 1/4 1/4 1/4]
result1 = sp.optimize.fmin_slsqp(flarray, w_init, bounds=[(0,1), (0,1), (0,1) , (0,1)], eqcons=[eq_constraint])

[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734254531668 분자
0.003907724857452827 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734254531668 분자
0.003907724857452827 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734422823467 분자
0.00390772508115553 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734388836715 분자
0.003907724744004626 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734407792398 분자
0.003907724833312011 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734393214582 분자
0.003907725004257699 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9013059461236332 분자
0.009045178996442641 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.930685279187818

[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9534188077612346 분자
0.0020440487323800647 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9733787656497624 분자
0.0006815458543232085 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9788376871001323 분자
0.0005191240658779502 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9788376871001323 분자
0.0005191240658779502 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9788377039293124 분자
0.0005191240741254994 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9788377005306372 분자
0.0005191240724244036 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9788377024262054 분자
0.0005191240743879998 분모
[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852

In [ ]:
result1

In [22]:
flarray(result1)

[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.979819365838447 분자
0.0005081432775386836 분모


-1928.2344353435944

In [23]:
flarray([0.25, 0.25, 0.25, 0.25])

[1.1293871479571596, 0.901305916305916, 1.0285153583617728, 0.9306852791878185] 초과수익률
0.9974734254531668 분자
0.003907724857452827 분모


-255.25682125515613